<h1> Try it yourself notebook: The Query Refiner </h1>
<p> In this notebook you will go through all the steps of making a simple agentic AI application using Langgraph. This notebook isn't complete and requires YOU to write some code. </p>
<p> This AI system has the intention of retrieving a messy (bad formulated) user query, transform it into a well-formulated one and finally, answer it. </p>
<h2> 1 Import packages and load enviroment variables </h2>

In [ ]:
# Enviroment 
import os

# Classes packages
from typing import Annotated
from typing import TypedDict
from pydantic import BaseModel, Field

# "Lang" packages
import langchain
import langgraph
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.prompts import ChatPromptTemplate
from langgraph.checkpoint.memory import MemorySaver

from IPython.display import Image, display

# .env variables
# my_api_key = os.getenv("OPENAI_API_KEY")

<h1> 2 State </h1>
<p> The class State is the state of the system. You can look at it as the memory of what the chatbot-system remebers.</p>

In [2]:
class State(TypedDict):
    messages: Annotated[list, add_messages]
    refined_q: str = Field(description="A refined version of the latest user query.")

<h2> 3 Nodes </h2>
<p> In this task you shall do the following: 
    <ol>
        <li> Come up with a system prompt that transform a messy user query into a well defined question. </li>
        <li>Determine which variable that shall be sent into the state of the system</li>
        <li>Create a prompt template and a chain.</li>
    </ol>
</p>

In [13]:
llm = ChatOpenAI(
    model_name="gpt-4o-mini",
    temperature=0,
    api_key=my_api_key
)


def refine_node(state: State):
    # Fill in the blank
    system_prompt = None
    prompt = ChatPromptTemplate.form_messages([
        ("system", system_prompt)
    ])
    chain = prompt | llm
    response = chain.invoke({"user": state["messages"]}).content
    # Fill in the blank
    return {"refined_question": None}


def answer_node(state:State):
    system_prompt = """
                    Answer the following user question. Be brief, but concise!
                    """
    # Fill in the blanks
    prompt = None
    chain = None
    response = chain.invoke({"user": state["messages"]}).content
    return {"messages": response}

<h2> 4 Edges </h2>
<p> Since we do not have any optional paths that the graph can take, we do not need to specify any conditional edges. </p>
<br>
<h2> 5 Compile workflow </h2>
<p> In the code cell below you have two tasks: 
    <ol>
        <li> Give a name to a node and define which function it shall utilize.</li>
        <li> Specify the start and end nodes for two edges.</li>
    </ol> 
</p>

In [ ]:
workflow = StateGraph(State)
memory = MemorySaver()

workflow.add_node("refine", refine_node)
# Fill in the blanks
workflow.add_node(None, None)

workflow.add_edge(START, "refine")
# Fill in the blanks
workflow.add_edge(None,None)
workflow.add_edge(None, END)

mygraph = workflow.compile(checkpointer=memory)

<p> Get an visual representation of the graph you just compiled. </p>

In [ ]:
try:
    display(Image(mygraph.get_graph().draw_mermaid_png()))
except Exception:
    pass

<h2> 6  Start chatting </h2>
<p> Run the cell below without editing, and start chatting with your agentic AI-application </p>

In [ ]:
def stream_graph_updates(user_input: str):
    for event in mygraph.stream({"messages": [("user", user_input)]},
                                {"configurable": {"thread_id": "123abc"}}):
        for value in event.values():
            print("Assistant:", value)


while True:
    try:
        user_input = input("User: ")
        print(user_input)
        if user_input.lower() in ["quit", "exit", "q"]:
            print("Goodbye!")
            break

        stream_graph_updates(user_input)
    except:
        # fallback if input() is not available
        user_input = "----Something wong here------"    
        print("User: " + user_input)
        stream_graph_updates(user_input)
        break

<h2> Finale </h2>
<p> Congratulations, you have just completed this notebook "tutorial"!</p>
<br>
<p> You are now free to either continue building functionality on this graph, or start from scratch and build your own one. There are many possibilities with Langgraph </p>
<br>
<p>
    <b>Recommened reading/tutorials:</b>
        <ul>
            <li><a href="https://python.langchain.com/docs/introduction/"> Langchain</a>, building chatbots. </li>
            <li><a href="https://langchain-ai.github.io/langgraph/tutorials/introduction/">Langgraph</a>, building agentic systems </li>
            <li><a href="https://docs.smith.langchain.com/">Langsmith</a>, monitor graph actions. </li>
        </ul>
</p>